In [ ]:
import numpy as np # Linear Algebra
import matplotlib.pyplot as plt # Plotting

In [ ]:
# We will define a quadratic objective function and a cubic constraint
Q = np.array([[2., 0.5], [0.5, 1.]])
x_target = np.array([[1.], [1.]])

def f(x):
    x = np.asarray(x).reshape((2, -1))
    return (0.5*(x - x_target).T @ Q @ (x - x_target))[0, 0]

def df(x):
    x = np.asarray(x).reshape((2,-1))
    return Q @ (x - x_target)

def ddf(x):
    return Q

In [ ]:
def h0(x):
    return x**3 + 0.5 * x**2 - 3. * x

def h(x):
   x = np.asarray(x).reshape((2,-1))
   return h0(x[0, 0]) - x[1, 0]

def dh(x):
    x = np.asarray(x).reshape((2,-1))
    return np.asarray([3. * x[0, 0]**2 + x[0, 0] - 3., -1.]).reshape((1, 2))

def ddh(x, l = np.ones((1, 1))):
    x = np.asarray(x).reshape((2,-1))
    l = np.asarray(l).reshape((1,-1))
    h = l[0, 0] * np.array([[6. * x[0, 0] + 1., 0.], [0., 0.]])
    return h

In [ ]:
# Newton's Method with Equality Constraints
def constrained_newton_step(x0, l0):
    N = 2
    M = 1
    x0 = np.asarray(x0).reshape((N, -1))
    l0 = np.asarray(l0).reshape((M, -1))

    # Let's compute the KKT system
    H = ddf(x0) + ddh(x0, l0) # full Newton: we include the Hessian of the constraints
    C = dh(x0)

    kkt_system = np.zeros((N+M, N+M))
    kkt_system[:N, :N] = H
    kkt_system[:N, N:] = C.T
    kkt_system[N:, :N] = C

    # Let's compute the target vector
    v = np.zeros((N+M, 1))
    v[:N, :] = -df(x0) - C.T @ l0
    v[N:, :] = -h(x0)

    # Let's solve for Δz
    Delta = np.linalg.solve(kkt_system, v)

    # Decompose delta to Δx and Δλ
    DeltaX = Delta[:N, 0].reshape((N, -1))
    DeltaL = Delta[N:, 0].reshape((M, -1))

    return x0 + DeltaX, l0 + DeltaL

In [ ]:
# Gauss-Newton with Equality Constraints
def gauss_newton_step(x0, l0):
    N = 2
    M = 1
    x0 = np.asarray(x0).reshape((N, -1))
    l0 = np.asarray(l0).reshape((M, -1))

    # Let's compute the KKT System
    H = ddf(x0) # We drop the second derivative of the constraints!
    C = dh(x0)

    kkt_system = np.zeros((N+M, N+M))
    kkt_system[:N, :N] = H
    kkt_system[:N, N:] = C.T
    kkt_system[N:, :N] = C

    # Let's compute the target vector
    v = np.zeros((N+M, 1))
    v[:N, :] = -df(x0) - C.T @ l0
    v[N:, :] = -h(x0)

    # Let's solve for Δz
    Delta = np.linalg.solve(kkt_system, v)

    # Decompose delta to Δx and Δλ
    DeltaX = Delta[:N, 0].reshape((N, -1))
    DeltaL = Delta[N:, 0].reshape((M, -1))

    return x0 + DeltaX, l0 + DeltaL

In [ ]:
# Let's plot things
plt.close() # close previous
fig = plt.figure()
ax = fig.add_subplot(111)

N = 40
x1 = np.linspace(-4., 4., N)
x2 = np.linspace(-4., 4., N)

X, Y = np.meshgrid(x1, x2)

val = np.zeros((N, N))
for i in range(N):
    for j in range(N):
        xx = np.zeros((2, 1))
        xx[0] = X[i, j]
        xx[1] = Y[i, j]
        val[i, j] = f(xx)

CS = ax.contour(x1.reshape((N,)), x2.reshape((N,)), val)

x = np.linspace(-2.4, 2., 1000)
y = h0(x)
ax.plot(x, y, label='c(x)', color='orange');

In [ ]:
# Initial point!
x_init = [0.9, -1.5] # Works OK for both!
# x_init = [-1., -3.] # Breaks for full Newton
# x_init = [-1.2, 2.5] # Breaks for full Newton, local solution for Gauss-Newton
l_init = [0.]
ax.plot(x_init[0], x_init[1], 'rx')

# Optimization
x_new = np.copy(x_init)
l_new = np.copy(l_init)

fig # show figure again with updated point(s)

In [ ]:
x_new, l_new = constrained_newton_step(x_new, l_new)
# x_new, l_new = gauss_newton_step(x_new, l_new)
ax.plot(x_new[0], x_new[1], 'rx')

print("x =", x_new.T, "f(x) =", f(x_new), "h(x) =", h(x_new))

fig # show figure again with updated point(s)